In [ ]:
#!pip install transformers torchaudio


In [ ]:
#!pip install pyctcdecode
#!pip install https://github.com/kpu/kenlm/archive/master.zip
#import kenlm  
#import pyctcdecode.decoder  
#pyctcdecode.decoder.kenlm = kenlm 
#restart session after this plzzzzzzz

In [52]:
import torch
import librosa


from transformers import AutoProcessor, AutoModelForCTC

processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-french")
model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-french")

In [53]:
print(model)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [55]:

speech, sr = librosa.load("/kaggle/input/dataaa2/test.mp3", sr=16000)
#must be 16000 cause it says "When using this model, make sure that your speech input is sampled at 16kHz."
input_values = processor(speech, return_tensors="pt", sampling_rate=16000,padding=True).input_values
with torch.no_grad():
    logits = model(input_values).logits
predicted_ids = torch.argmax(logits,dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(transcription[0])

quel queltierpètes et icaine suwicem lisansale au cinima fatait super et aussi eetaleàlaatregan retoue de cotug jeles voir en film et pus afachelé occu parpidoar atrofie au charlevor des photos e juverant la fontion profensuchi so catément sugé dan rowens bedi afajonc 'encroet à trolo


### bad bad bad model above , good below

In [57]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)


audio_array, sr = librosa.load("/kaggle/input/dataaa2/test.mp3", sr=16000)

sample = {"array": audio_array, "sampling_rate": sr}

result = pipe(sample)
print(result["text"])


Device set to use cuda:0
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Qu'est-ce que tu as fait ce week-end ? Ce week-end nous sommes allés au cinéma, c'était super cool et aussi on est allés à le parc. Très bien. Tu vois, tu es allé quoi ce week-end ? Je suis allé voir un film et puis après je suis allé au que par. Et toi ? Tranquille, moi je suis allé voir des potos, tu vois, on a fait une petite bouffe à un sushi, tu vois. Tu fais tellement le sushi toi ? Ouais ouais, ça te bête là. Franchement c'était incroyable, trolo.


### good enaugh now with the pheneomeme thing

In [60]:
#!pip install phonemizer

In [59]:
#the model that does the phenomeosing 
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("Cnam-LMSSC/wav2vec2-french-phonemizer")
model = Wav2Vec2ForCTC.from_pretrained("Cnam-LMSSC/wav2vec2-french-phonemizer")

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

In [62]:
#print(model)

In [71]:
import numpy as np

In [72]:
input_values=processor(np.array(audio_array),sampling_rate=16_000., return_tensors="pt")
with torch.no_grad():
  logits = model(**input_values).logits

predicted_ids = torch.argmax(logits,dim = -1)
transcription = processor.batch_decode(predicted_ids)

print("Phonetic transcription : ", transcription)

Phonetic transcription :  ['ɛkɛkkɛdəɛm syvikɛn lisɑzalɛ osnimafɛtɛ sipapy e osi ɑan alaka tʁɛbjɔ̃ ʁəs də kɔsvi ʒle vwaʁ ɑ̃film py apʁoʃle ɔkypa edwa atʁʃe maʃyʁ le ṽ de podo syvɑ̃ la fimkufɑɑ̃ syʃi kətamasd woɛsɑ̃ bɛ̃da apəʒustœ̃ koʁɛa pʁolo']


In [ ]:
#for the prompt we focus on the top 3 pointss kids struggle with 
#when speaking french so it is what the feedback 
#strucure focuses on
#1 nasal vowels
# 2.Over-articulation or English/Arabic rhythm
#3.Liaison and Final Consonant Rules

In [ ]:
#on the side there is always popup that give different prompt to parent //different feeback to partn fi wa9too
#so feedback to both parent and child 
#chech it fi wa9too walla hana it is saved in resumt of what you child did today

In [90]:
promptparent = f"""
🎧 The following is a child's spoken sentence along with their phonetic transcription. Based on this and the expected French pronunciation, provide clear, structured feedback focusing on:

1. Nasal vowels  
2. Over-articulation or non-French rhythm (e.g., English or Arabic)  
3. Liaison and final consonant rules  

For each category, indicate:

✔️ What the child did well  
⚠️ What needs improvement (with examples)  
🎯 A suggestion or tip to improve  

Also, finish with a brief overall clarity score (e.g., from 1–5 stars).

---

📝 **Written sentence (child was trying to say):**  
{result["text"]}

🔤 **Phonetic transcription (child's speech):**  
{transcription}
"""


In [91]:
promptchild = f"""🎧 The following is a child's spoken sentence along with their phonetic transcription. Based on this and the expected French pronunciation, provide simple easy to understand explanation to the child focusing on:
1. Nasal vowels
2. Over-articulation or non-French rhythm (e.g., English or Arabic)
3. Liaison and final consonant rules

For each category, Explain to the child (like you are talking to him):

✔️ What the child did well  
⚠️ What needs improvement (with examples)  
🎯 A suggestion or tip to improve

Also, finish with a brief overall clarity score (e.g., from 1–5 stars).
📝 **Written sentence (child was trying to say):**  :${result["text"]},
🔤 **Phonetic transcription (child's speech):** :
${transcription}
"""


In [92]:
from google import genai

client = genai.Client(api_key=API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=promptchild,
)

print(response.text)

Okay, let's listen to your French! You're doing great trying to speak French, it's a tricky language! Here's what I noticed:

**1. Nasal Vowels: Those Sounds Coming From Your Nose!**

✔️ **What you did well:** You tried some nasal sounds! I heard little bits of nasalization, so you're starting to get it.

⚠️ **What needs improvement:** Remember those sounds that feel like they're coming from your nose and mouth at the same time? It sounds like you're sometimes forgetting to make your nose vibrate! For example:

*   When you said "un film," the "un" should have a nasal "uh" sound.
*   When you said "Très bien," the "bien" should have a nasal "ah" sound.
* You seem to also struggle with nasal vowels in words like "cinéma"

🎯 **Suggestion/Tip:** Try pinching your nose while saying "on". Now, release your nose while still saying "on". You'll feel the vibration of air going through your nose. Do this often to learn to create the nasal vowel in "on". There are other nasal vowels in French, e